In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

# Initialize client
client = bigquery.Client(project='rock-finder-project')

In [ ]:
import pandas_gbq

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sql = "SELECT * FROM rock-finder-project.routes.mp_routes"
p_id="rock-finder-project"
df_routes = pandas_gbq.read_gbq(sql, project_id=p_id)
df_routes


In [ ]:
#Null value in Rating
df_routes[['Rating']].isnull().sum()

In [ ]:
# How many 'Alpine' in Route Type
df_routes['Route Type'].value_counts()